In [1]:
from google.colab import files
uploaded = files.upload()

Saving clust_data.csv to clust_data.csv
Saving recommendation_mini.csv to recommendation_mini.csv


In [7]:
import pandas as pd

In [10]:
# prompt: Dataset Information: cluster_data.csv
#  The experiments have been carried out with a group of 30 volunteers within an age bracket
#  of 19-48 years. Each person performed six activities (WALKING, WALKING_UPSTAIRS,
#  WALKING_DOWNSTAIRS,SITTING, STANDING, LAYING) wearing a smartphone
#  (Samsung Galaxy S II) on the waist. Using its embedded accelerometer and gyroscope, they
#  have captured 3-axial linear acceleration and 3-axial angular velocity at a constant rate of
#  50Hz. The experiments have been video-recorded to label the data manually.
#  This Dataset consist of
#  A561-feature vector with time and frequency domain variables.
#  Its activity label.
#  An identifier of the subject who carried out the experiment
#  Dataset Information: recommendation.csv
#  The recommendation.csv file consists of 85724 ratings given by 943 users on 1659
#  products.
#  akshaypurushan@gmail.com
#  CQAR0OSH88
#  It has the following 4 columns:
#  UserID
#  ItemID
#  Rating (Integers 1 to 5)
#  Timestamp (Unix time stamp).
#  Note:
#  1.Use cluster_data.csv for all the clustering and dimensionality reduction questions
#  2.Use recommendations.csv for recommendation system questions
#  3.'activity' column in the cluster_data.csv is the target column. Don't use this column for
#  clustering purposes. You can use this for predictive model building

import pandas as pd

# Load the datasets
cluster_data = pd.read_csv('clust_data.csv')
recommendation_data = pd.read_csv('recommendation_mini.csv')

# Display the first few rows and information about each dataset
print("Cluster Data:")
print(cluster_data.head())
print("\nCluster Data Info:")
cluster_data.info()

print("\nRecommendation Data:")
print(recommendation_data.head())
print("\nRecommendation Data Info:")
recommendation_data.info()

Cluster Data:
          F1         F2          F3         F4          F5          F6  \
0  -3.026950  78.111415 -374.114333  70.234478   47.055428 -132.530866   
1  10.666065  38.688616    0.982882 -90.163689 -209.167839  -15.156554   
2  12.889724   6.537806   -7.172982  13.354590 -104.688780   14.148796   
3 -13.518929  93.875045  -93.384710 -20.934313  -21.779267   21.189977   
4  -1.892682  -0.630952  -61.208627 -34.865905  -12.535872  -12.747138   

           F7          F8          F9         F10  ...         F12        F13  \
0 -650.397369  209.966341    9.216498  152.958095  ... -116.980754  21.669078   
1  -64.164883  157.497306 -147.046024  443.664845  ... -214.024783 -25.270747   
2  -47.564563  -83.617770 -127.093760 -155.737080  ...   69.016811 -17.877942   
3   97.753321  -71.051340   89.128396  -80.578483  ...   49.643950 -11.838531   
4  -95.405479  -13.823230  -48.152023   95.988291  ...   89.092312   2.923746   

         F14         F15         F16        F17       

In [8]:
# prompt:  Perform required pre-processing and compute how many pairs of variables have the
#  correlation more than 0.8 ? Apply PCA and compute the required number of principal
#  components to capture the 90 percent variance of the original data. Print the Eigenvalues and
#  Eigenvectors of top 5 PCs

import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Pre-processing the recommendation_data for correlation and PCA
# Assuming numerical columns are relevant for correlation and PCA
recommendation_numeric = recommendation_data.select_dtypes(include=np.number)

# Drop columns with missing values or handle them appropriately
# For simplicity, dropping columns with any missing values for demonstration
recommendation_numeric_cleaned = recommendation_numeric.dropna(axis=1)

# Compute the correlation matrix
correlation_matrix = recommendation_numeric_cleaned.corr()

# Find pairs with correlation > 0.8 (excluding self-correlation)
high_corr_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i + 1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.8:
            high_corr_pairs.append((correlation_matrix.columns[i], correlation_matrix.columns[j]))

print(f"\nNumber of pairs with correlation > 0.8: {len(high_corr_pairs)}")
# print("High correlation pairs:", high_corr_pairs) # Uncomment to see the pairs

# Scale the data before applying PCA
scaler = StandardScaler()
scaled_data = scaler.fit_transform(recommendation_numeric_cleaned)

# Apply PCA
pca = PCA()
pca.fit(scaled_data)

# Compute the required number of components to capture 90% variance
cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
n_components_90 = np.where(cumulative_variance >= 0.90)[0][0] + 1

print(f"\nNumber of principal components to capture 90% variance: {n_components_90}")

# Print Eigenvalues (explained variance) of top 5 PCs
print("\nEigenvalues (Explained Variance) of top 5 PCs:")
for i in range(min(5, len(pca.explained_variance_))):
    print(f"PC {i+1}: {pca.explained_variance_[i]:.4f}")

# Print Eigenvectors (components) of top 5 PCs
print("\nEigenvectors of top 5 PCs:")
# Each row of components_ corresponds to a principal component
# Each column of components_ corresponds to an original feature
for i in range(min(5, len(pca.components_))):
    print(f"PC {i+1}: {pca.components_[i]}")

# Optional: Plot cumulative explained variance
plt.figure(figsize=(8, 4))
plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, marker='o', linestyle='--')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance Ratio')
plt.title('Cumulative Explained Variance by Number of Components')
plt.axhline(y=0.90, color='r', linestyle='-', label='90% Variance')
plt.axvline(x=n_components_90, color='g', linestyle='-', label=f'{n_components_90} Components')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# prompt: Build the K-means clustering model with reduced PCA features (PCs which are explaining 90
#  percent variance) and compute the optimal value of clusters. Make the business inferences
#  using the cluster groups.

import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Apply PCA with the number of components explaining 90% variance
pca_final = PCA(n_components=n_components_90)
pca_data = pca_final.fit_transform(scaled_data)

print(f"\nShape of PCA reduced data: {pca_data.shape}")

# Determine the optimal number of clusters using the Elbow Method and Silhouette Score
# Using Elbow Method
inertia = []
for k in range(1, 11): # Test k from 1 to 10
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(pca_data)
    inertia.append(kmeans.inertia_)

plt.figure(figsize=(8, 4))
plt.plot(range(1, 11), inertia, marker='o')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Inertia')
plt.title('Elbow Method for Optimal k')
plt.xticks(range(1, 11))
plt.grid(True)
plt.show()

# Using Silhouette Score (for k > 1)
silhouette_scores = []
for k in range(2, 11): # Silhouette score requires at least 2 clusters
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(pca_data)
    score = silhouette_score(pca_data, kmeans.labels_)
    silhouette_scores.append(score)

plt.figure(figsize=(8, 4))
plt.plot(range(2, 11), silhouette_scores, marker='o')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score for Optimal k')
plt.xticks(range(2, 11))
plt.grid(True)
plt.show()

# Based on the Elbow and Silhouette plots, choose an optimal k
# Let's assume from the plots, k=3 or k=4 appears optimal for demonstration
optimal_k = 3 # Replace with the value you determine from the plots

print(f"\nChoosing optimal number of clusters: {optimal_k}")

# Build K-means model with the optimal k
kmeans_optimal = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
clusters = kmeans_optimal.fit_predict(pca_data)

# Add the cluster labels back to the original cleaned recommendation data
recommendation_numeric_cleaned['Cluster'] = clusters

# Analyze the characteristics of each cluster
print("\nCluster Analysis:")
for cluster_id in range(optimal_k):
    cluster_data = recommendation_numeric_cleaned[recommendation_numeric_cleaned['Cluster'] == cluster_id]
    print(f"\nCluster {cluster_id} (Number of data points: {len(cluster_data)}):")
    print(cluster_data.drop('Cluster', axis=1).describe().T) # Describe the numerical features in the cluster

# Business Inferences based on cluster analysis
# The interpretation here depends heavily on the actual features in your 'recommendation_numeric_cleaned' data.
# Replace the comments below with actual interpretations based on your data's columns and the .describe() output.

print("\nBusiness Inferences:")
# Example: If your data contains purchase frequency, average order value, etc.
# Based on the .describe() output for each cluster, you can infer:

# Cluster 0: (Example) Might represent high-value customers - high purchase frequency, high average order value.
# Potential Business Actions: Offer loyalty programs, personalized high-end product recommendations, dedicated support.

# Cluster 1: (Example) Might represent new or infrequent customers - low purchase frequency, lower average order value.
# Potential Business Actions: Targeted promotions to encourage repeat purchases, onboarding campaigns, product education.

# Cluster 2: (Example) Might represent bargain hunters - moderate purchase frequency, low average order value, focused on discounted items.
# Potential Business Actions: Send notifications about sales and discounts, bundle deals, promote value-for-money products.

# You would need to look at the means/medians/standard deviations of the original scaled or unscaled features within each cluster
# (by describing the cluster_data) to make specific inferences relevant to your business and data.
# For a more robust analysis, you might also consider analyzing categorical features if they were included initially.

In [ ]:
# prompt: Build/Plot the top 100 cluster dendogram using 4 different linkages and compare its
#  performance.

import matplotlib.pyplot as plt
import numpy as np
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import pdist

# Use the PCA reduced data for hierarchical clustering
# We will work with a sample of the data for plotting the dendrogram
# as plotting for 100 samples is more manageable for visualization.
# Let's sample 100 data points from the PCA data.
np.random.seed(42) # for reproducibility
sample_size = min(100, pca_data.shape[0])
sample_indices = np.random.choice(pca_data.shape[0], size=sample_size, replace=False)
pca_data_sample = pca_data[sample_indices]

# Calculate the distance matrix
distance_matrix = pdist(pca_data_sample)

# Define different linkage methods
linkage_methods = ['ward', 'average', 'complete', 'single']

plt.figure(figsize=(15, 10))

for i, method in enumerate(linkage_methods):
    plt.subplot(2, 2, i + 1)
    # Perform hierarchical clustering using the current linkage method
    linked = linkage(distance_matrix, method)

    # Plot the dendrogram
    dendrogram(linked, orientation='top', distance_sort='descending', show_leaf_counts=True)
    plt.title(f'Dendrogram (Linkage: {method}) - Top {sample_size} Samples')
    plt.xlabel('Sample Index')
    plt.ylabel('Distance')

plt.tight_layout()
plt.show()

# Performance Comparison (Qualitative based on Dendrogram Visualization)
print("\nPerformance Comparison of Linkage Methods (Qualitative based on Dendrogram):")
print("- Ward: Tends to produce compact, spherical clusters. Minimizes the variance of the clusters being merged.")
print("- Average: Uses the average distance between all pairs of observations between the two sets of data. Can produce more balanced trees than single linkage.")
print("- Complete: Uses the maximum distance between all pairs of observations between the two sets of data. Tends to produce more compact clusters, but sensitive to outliers.")
print("- Single: Uses the minimum distance between all pairs of observations between the two sets of data. Tends to produce long, 'straggly' clusters (chaining effect), sensitive to noise.")
print("\nChoosing the best linkage method depends on the structure of the data and the desired cluster properties.")
print("Visual inspection of the dendrograms helps in understanding how clusters are formed at different distance thresholds.")
print("For quantitative comparison, one might use metrics like cophenetic correlation coefficient (though not explicitly shown in the plot itself).")
print("However, the request was to compare performance based on plotting the dendrogram, which is primarily a visual assessment of how clearly separated and meaningful the clusters appear at different linkage levels.")


In [ ]:
# prompt:  Clusterthedatainto5groupsusingK-meansandordertheclustersintermsofthe
#  inertia(WCSS)ofeachcluster.

import numpy as np
# Clusterthedatainto5groupsusingK-means
n_clusters_kmeans = 5
kmeans_5 = KMeans(n_clusters=n_clusters_kmeans, random_state=42, n_init=10)
kmeans_5.fit(pca_data)
clusters_5 = kmeans_5.labels_

# Get the inertia (WCSS) for each cluster
# We can access the inertia for the overall clustering, but to get WCSS for each cluster,
# we need to calculate it manually based on the assigned points and cluster centroids.
# The inertia_ attribute of the KMeans model gives the sum of squared distances of samples
# to their closest cluster center for *all* clusters.

# Calculate WCSS for each cluster manually
cluster_inertia = []
for i in range(n_clusters_kmeans):
    # Get data points belonging to this cluster
    cluster_points = pca_data[clusters_5 == i]
    # Get the centroid for this cluster
    cluster_centroid = kmeans_5.cluster_centers_[i]
    # Calculate the sum of squared distances from points to the centroid
    inertia_i = np.sum((cluster_points - cluster_centroid) ** 2)
    cluster_inertia.append((i, inertia_i))

# Order the clusters by inertia (WCSS) in ascending order
ordered_clusters = sorted(cluster_inertia, key=lambda item: item[1])

print(f"\nWCSS (Inertia) for each cluster (ordered by WCSS):")
for cluster_id, inertia_value in ordered_clusters:
    print(f"Cluster {cluster_id}: {inertia_value:.2f}")

# Add the 5-cluster labels to the dataframe (optional, but often useful)
# recommendation_numeric_cleaned['Cluster_5'] = clusters_5
# print("\nRecommendation Data with 5-Cluster Labels:")
# print(recommendation_numeric_cleaned.head())

In [ ]:
# prompt: BuildthefollowingMLmodelandcompareitsperformance:
#  a.MLmodelwithoriginalinp_dataandout
#  b.MLmodelwithpca_inp_dataandoutput
#  c.MLmodelwithsvd_inp_dataandout
#  d.MLmodelwithlda_inp_dataandout
#  Note1:The‘activity’columninthedatasetistheoutputcolumn(out)
#  Note2:
#  inp_data▯Allthecolumnsintheoriginaldataset(excluding‘activity’)
#  pca_inp_data▯number of PCAcomponentswhich captures the 95 percent of
#  variancesvd_inp_data▯numberofSVDcomponentswhichcapturesthe95percent
#  ofvariancelda_inp_data▯requirednumberofLDAcomponents

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.decomposition import TruncatedSVD
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import time

# Assuming recommendation_data is already loaded and cleaned as recommendation_numeric_cleaned

# Drop non-numeric columns if any were introduced later (like 'Cluster')
# Re-select only the original numeric columns for consistency before splitting
# Make sure 'activity' column is present in the original 'recommendation_data' and is the output
# If 'activity' is not in recommendation_data but is in cluster_data, we need to adjust.
# Let's assume 'activity' is the target variable and is in the original recommendation_data.
# We will need to merge or ensure 'activity' is available.

# Re-load the recommendation data to ensure we have the 'activity' column
# assuming 'activity' is in the original 'recommendation_mini.csv'
try:
    recommendation_data_full = pd.read_csv('recommendation_mini.csv')
except FileNotFoundError:
    print("Error: 'recommendation_mini.csv' not found. Please ensure it is uploaded or in the correct path.")
    # Exit or handle the error appropriately
    exit()

# Separate the target variable ('activity')
if 'activity' in recommendation_data_full.columns:
    out = recommendation_data_full['activity']
    inp_data_full = recommendation_data_full.drop('activity', axis=1)
else:
    print("Error: 'activity' column not found in 'recommendation_mini.csv'. Cannot proceed.")
    # Exit or handle the error appropriately
    exit()

# Select only numeric columns from the input features
inp_data_numeric = inp_data_full.select_dtypes(include=np.number)

# Handle missing values in the input data if any (e.g., imputation or dropping)
# For simplicity, we'll drop columns with missing values again, consistent with previous steps.
inp_data = inp_data_numeric.dropna(axis=1)

# Ensure the target variable 'out' corresponds to the rows in 'inp_data' after dropping NaNs
# We need to align 'out' with the rows that remained in 'inp_data'.
# Since we dropped columns, the rows should still align. If we dropped rows, we'd need to filter 'out'.
# Assuming dropping columns doesn't change the row order/indices, this alignment is fine.

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(inp_data, out, test_size=0.2, random_state=42)

# Scale the data (important for PCA, SVD, LDA, and many models)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --- a. ML model with original inp_data and out ---
print("\n--- Training ML Model with Original Data ---")
start_time = time.time()
model_original = LogisticRegression(max_iter=1000) # Increased max_iter for potential convergence issues
model_original.fit(X_train_scaled, y_train)
y_pred_original = model_original.predict(X_test_scaled)
accuracy_original = accuracy_score(y_test, y_pred_original)
report_original = classification_report(y_test, y_pred_original)
end_time = time.time()
print(f"Training time: {end_time - start_time:.4f} seconds")
print(f"Accuracy: {accuracy_original:.4f}")
print("Classification Report:")
print(report_original)


# --- Determine number of components for PCA and SVD (95% variance) ---
# Fit PCA on the scaled training data to find components for 95% variance
pca_for_components = PCA()
pca_for_components.fit(X_train_scaled)
cumulative_variance = np.cumsum(pca_for_components.explained_variance_ratio_)
# number of PCA components which captures the 95 percent of variance
n_components_pca = np.where(cumulative_variance >= 0.95)[0][0] + 1
print(f"\nNumber of PCA components for 95% variance: {n_components_pca}")

# Fit SVD on the scaled training data to find components for 95% variance
svd_for_components = TruncatedSVD(n_components=X_train_scaled.shape[1] - 1) # Max components is n_features - 1 or min(n_samples, n_features)
svd_for_components.fit(X_train_scaled)
cumulative_variance_svd = np.cumsum(svd_for_components.explained_variance_ratio_)
# number of SVD components which capture the 95 percent of variance
# Need to handle cases where 95% variance is not reached even with max components
n_components_svd = X_train_scaled.shape[1] - 1
if np.any(cumulative_variance_svd >= 0.95):
     n_components_svd = np.where(cumulative_variance_svd >= 0.95)[0][0] + 1
else:
    print("Warning: 95% variance not captured by SVD with max components. Using max possible components.")

print(f"Number of SVD components for 95% variance: {n_components_svd}")

# --- Determine number of components for LDA ---
# LDA's number of components is min(n_classes - 1, n_features)
n_classes = len(np.unique(y_train))
# required number of LDA components
n_components_lda = min(n_classes - 1, X_train_scaled.shape[1])
print(f"Required number of LDA components: {n_components_lda}")

# --- b. ML model with pca_inp_data and output ---
print("\n--- Training ML Model with PCA Reduced Data ---")
# Apply PCA with the determined number of components
pca_final = PCA(n_components=n_components_pca)
X_train_pca = pca_final.fit_transform(X_train_scaled)
X_test_pca = pca_final.transform(X_test_scaled)

start_time = time.time()
model_pca = LogisticRegression(max_iter=1000)
model_pca.fit(X_train_pca, y_train)
y_pred_pca = model_pca.predict(X_test_pca)
accuracy_pca = accuracy_score(y_test, y_pred_pca)
report_pca = classification_report(y_test, y_pred_pca)
end_time = time.time()
print(f"Training time: {end_time - start_time:.4f} seconds")
print(f"Accuracy: {accuracy_pca:.4f}")
print("Classification Report:")
print(report_pca)


# --- c. ML model with svd_inp_data and out ---
print("\n--- Training ML Model with SVD Reduced Data ---")
# Apply SVD with the determined number of components
svd_final = TruncatedSVD(n_components=n_components_svd, random_state=42)
X_train_svd = svd_final.fit_transform(X_train_scaled)
X_test_svd = svd_final.transform(X_test_scaled)

start_time = time.time()
model_svd = LogisticRegression(max_iter=1000)
model_svd.fit(X_train_svd, y_train)
y_pred_svd = model_svd.predict(X_test_svd)
accuracy_svd = accuracy_score(y_test, y_pred_svd)
report_svd = classification_report(y_test, y_pred_svd)
end_time = time.time()
print(f"Training time: {end_time - start_time:.4f} seconds")
print(f"Accuracy: {accuracy_svd:.4f}")
print("Classification Report:")
print(report_svd)


# --- d. ML model with lda_inp_data and out ---
print("\n--- Training ML Model with LDA Reduced Data ---")
# LDA requires the target variable for fitting
# Check if LDA is applicable (n_components_lda > 0 and n_samples >= n_classes)
if n_components_lda > 0 and X_train_scaled.shape[0] >= n_classes:
    try:
        # Apply LDA with the determined number of components
        lda_final = LinearDiscriminantAnalysis(n_components=n_components_lda)
        X_train_lda = lda_final.fit_transform(X_train_scaled, y_train)
        X_test_lda = lda_final.transform(X_test_scaled)

        start_time = time.time()
        model_lda = LogisticRegression(max_iter=1000)
        model_lda.fit(X_train_lda, y_train)
        y_pred_lda = model_lda.predict(X_test_lda)
        accuracy_lda = accuracy_score(y_test, y_pred_lda)
        report_lda = classification_report(y_test, y_pred_lda)
        end_time = time.time()
        print(f"Training time: {end_time - start_time:.4f} seconds")
        print(f"Accuracy: {accuracy_lda:.4f}")
        print("Classification Report:")
        print(report_lda)
    except ValueError as e:
        print(f"Could not perform LDA: {e}")
        print("LDA might not be applicable due to data characteristics or insufficient samples per class.")
else:
    print("\n--- Skipping LDA ---")
    print(f"LDA is not applicable. Required LDA components: {n_components_lda}. Number of training samples: {X_train_scaled.shape[0]}. Number of classes: {n_classes}.")


# --- Comparison of Performance ---
print("\n--- Performance Comparison ---")
print(f"Model with Original Data Accuracy: {accuracy_original:.4f}")
if 'accuracy_pca' in locals():
    print(f"Model with PCA Data Accuracy: {accuracy_pca:.4f}")
if 'accuracy_svd' in locals():
    print(f"Model with SVD Data Accuracy: {accuracy_svd:.4f}")
if 'accuracy_lda' in locals():
     print(f"Model with LDA Data Accuracy: {accuracy_lda:.4f}")

print("\nInterpretation:")
print("- Compare the Accuracy scores for each model.")
print("- Compare the Classification Reports (Precision, Recall, F1-score) for each class.")
print("- Consider the training time for each model (especially relevant for larger datasets).")
print("- Dimensionality reduction techniques (PCA, SVD, LDA) can sometimes improve model performance (e.g., by reducing noise, multicollinearity) or reduce training time, but can also lead to information loss and decreased performance.")
print("- The best approach depends on the specific dataset and the characteristics of the features and target variable.")


In [ ]:
# prompt: Usethedataset:recommendation.csv
#  Buildthepopularitybasedrecommendationsystemandsuggesttop5items.

# Popularity-based recommendation system
# This approach recommends items that are most popular across all users.
# popularity is typically measured by the number of times an item appears
# in the dataset (e.g., number of ratings, purchases, etc.).

# Assuming 'recommendation_data' contains a column that represents items
# and implicit interactions (like presence implies interaction/interest).
# A simple approach is to count the occurrences of each item.

# Let's assume your recommendation_mini.csv has columns like 'user_id', 'item_id', etc.
# We need a column that identifies the items. Let's assume 'item_id' exists.
# If 'item_id' doesn't exist, you need to use a relevant item identifier column from your dataset.

# Check if 'item_id' column exists
if 'item_id' in recommendation_data.columns:
    # Calculate the popularity of each item by counting occurrences
    item_popularity = recommendation_data['item_id'].value_counts().reset_index()
    item_popularity.columns = ['item_id', 'popularity_count']

    # Sort items by popularity in descending order
    popular_items = item_popularity.sort_values(by='popularity_count', ascending=False)

    # Suggest the top 5 most popular items
    top_n = 5
    top_5_popular_items = popular_items.head(top_n)

    print(f"\n--- Top {top_n} Popular Items ---")
    print(top_5_popular_items)

else:
    print("\nError: 'item_id' column not found in the recommendation data.")
    print("Please ensure your 'recommendation_mini.csv' dataset has a column identifying items.")
    print("If your item identifier has a different column name, please update the code accordingly.")


In [ ]:
# prompt: Usethedataset:recommendation.csv
#  Buildacollaborativerecommendationenginetorecommendthetop5itemstothespecific
#  user.MeasurethemodelqualityintermsofRMSE

!pip install scikit-surprise
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

# Load the dataset
# Assuming recommendation_data_full is already loaded and contains 'user_id', 'item_id', and a rating/interaction column.
# If your dataset structure is different, adjust the column names below.
# Let's assume the interaction is represented by a rating or a value indicating interaction strength.
# If you only have user_id and item_id (implicit feedback), you might need to create a dummy rating.
# For this example, let's assume 'activity' is the rating/interaction column. If not, replace 'activity' with the correct column name.

# Define a Reader object. The rating_scale parameter is important.
# If 'activity' is a binary indicator (0 or 1), rating_scale should be (0, 1).
# If it's a rating scale (e.g., 1-5), set it accordingly.
# If it's implicit feedback, you might use a dummy rating of 1 for all interactions.
# Let's assume 'activity' is a measure of interaction strength, treat it as a rating.
# We need to check the range of the 'activity' column to set the rating_scale.
# Let's assume it's a positive value indicating interaction strength.

# Check the data types and range of 'activity' column
if 'activity' in recommendation_data_full.columns:
    activity_min = recommendation_data_full['activity'].min()
    activity_max = recommendation_data_full['activity'].max()
    print(f"\n'activity' column range: [{activity_min}, {activity_max}]")
    reader = Reader(rating_scale=(activity_min, activity_max))

    # Load the data from the pandas DataFrame
    # The order of columns in the DataFrame should be: user_id, item_id, rating
    # Ensure these columns exist in your recommendation_data_full dataframe.
    # If your user ID or item ID columns have different names, change 'user_id' and 'item_id'.
    try:
        data = Dataset.load_from_df(recommendation_data_full[['user_id', 'item_id', 'activity']], reader)
    except KeyError as e:
        print(f"\nError: Required column not found for collaborative filtering: {e}")
        print("Please ensure 'user_id', 'item_id', and 'activity' columns exist in 'recommendation_data_full'.")
        # Exit or handle the error appropriately
        exit()

    # Split data into training and testing sets for evaluating the model
    trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

    # Use the SVD algorithm for collaborative filtering
    # SVD is a matrix factorization technique commonly used in recommendation systems.
    algo = SVD(random_state=42)

    # Train the algorithm on the training set
    print("\nTraining SVD model...")
    algo.fit(trainset)
    print("Training complete.")

    # Make predictions on the test set
    print("Making predictions on test set...")
    predictions = algo.test(testset)
    print("Predictions complete.")

    # Measure the model quality in terms of RMSE
    rmse = accuracy.rmse(predictions)
    print(f"\nRMSE on the test set: {rmse:.4f}")

    # --- Recommendation for a specific user ---
    # Define the user ID you want recommendations for
    # Replace 'specific_user_id' with an actual user ID from your dataset
    # For demonstration, let's pick the first user ID from the training set
    specific_user_id = trainset.ir[0] # Get the internal user id of the first user in the training set
    # To get the original user ID, you can use trainset.to_raw_uid(specific_user_id)
    raw_specific_user_id = trainset.to_raw_uid(specific_user_id)

    print(f"\nGenerating recommendations for user: {raw_specific_user_id}")

    # Get a list of all unique items
    all_items = list(recommendation_data_full['item_id'].unique())

    # Get items the user has already interacted with in the training set
    # Convert raw user ID to internal user ID if needed, but predict expects raw IDs
    try:
        user_interacted_items = recommendation_data_full[recommendation_data_full['user_id'] == raw_specific_user_id]['item_id'].tolist()
    except KeyError:
         print(f"Warning: User ID {raw_specific_user_id} not found in the original recommendation_data_full for checking interacted items.")
         user_interacted_items = [] # Assume no prior interactions found if user ID is not in the original df

    # Find items the user has NOT interacted with
    items_to_predict = [item for item in all_items if item not in user_interacted_items]

    # Predict ratings for items the user has not interacted with
    # Create a list of (user_id, item_id, dummy_rating) tuples for prediction
    # Dummy rating is not used for prediction calculation itself, but required by predict function
    predictions_for_user = [(raw_specific_user_id, item, 0) for item in items_to_predict]

    # Get predicted ratings
    predicted_ratings = algo.test(predictions_for_user)

    # Sort predictions by estimated rating in descending order
    top_n = 5
    top_recommendations = sorted(predicted_ratings, key=lambda x: x.est, reverse=True)[:top_n]

    print(f"\nTop {top_n} recommendations for user {raw_specific_user_id}:")
    for recommendation in top_recommendations:
        # recommendation is a Prediction object: (uid, iid, r_ui, est, details)
        print(f"Item ID: {recommendation.iid}, Estimated Rating: {recommendation.est:.4f}")

else:
    print("\nError: 'activity' column not found in the recommendation data.")
    print("Please ensure your 'recommendation_mini.csv' dataset has a column representing the interaction strength (rating).")
    print("If your interaction column has a different name, please update the code accordingly.")
    print("If your data only has user and item IDs (implicit feedback), you may need to adjust the approach.")